In [2]:
import pandas as pd
import urllib
from urllib import request
import numpy as np
import sklearn 
from statistics import mean 
import statistics
from sklearn import preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import neural_network
from sklearn.neural_network import MLPRegressor
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder


In [3]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)

In [4]:
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name
df['weekStatus'] = df['date'].dt.dayofweek
df['weekendstatus'] = np.where(df['weekStatus'] < 5, 0, 1)

#To Calculate NSM
df['datenew'] = df['date'].dt.date
datenum=df['datenew'].unique()
lennsm = len(datenum)+1
#print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(df)]
df['NSM'] = dfc1
###################
df_day = pd.get_dummies(df['day_of_week'])
df = df.join(df_day)
del df['Monday']
del df['weekStatus']
del df['day_of_week']
del df['time']
del df['datenew']
#SETTING DATE COLUMN AS INDEX
df = df.set_index('date')

In [5]:
df.head(10)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.20,44.790000,19.79,44.730000,19.000000,45.566667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.20,44.722500,19.79,44.790000,19.000000,45.992500,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.20,44.626667,19.79,44.933333,18.926667,45.890000,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.20,44.590000,19.79,45.000000,18.890000,45.723333,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.20,44.530000,19.79,45.000000,18.890000,45.530000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,50,40,19.890000,46.026667,19.20,44.500000,19.79,44.933333,18.890000,45.730000,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,60,50,19.890000,45.766667,19.20,44.500000,19.79,44.900000,18.890000,45.790000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,60,50,19.856667,45.560000,19.20,44.500000,19.73,44.900000,18.890000,45.863333,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,60,40,19.790000,45.597500,19.20,44.433333,19.73,44.790000,18.890000,45.790000,...,18,11,0,66000,0,0,0,0,0,0


In [6]:
def mae(org,pred):
    return mean_absolute_error(org,pred)
def rmse(org,pred):
    return np.sqrt(mean_squared_error(org,pred))
def r2s(org,pred):
    return r2_score(org,pred)
def mape(org,pred):
    y_true, y_pred = np.array(org), np.array(pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [7]:
def scores(model,yorg,ypred):
    print("The scores for the model : ",model)
    print("MAE    :    ",mae(yorg,ypred))
    print("RMSE   :    ",rmse(yorg,ypred))
    print("R2     :    ",r2s(yorg,ypred))
    print("MAPE   :    ",mape(yorg,ypred))

Modeling with scaled data

In [8]:
a1 = df.Appliances.values
a2 = a1.reshape(len(a1),1)
scaler = MinMaxScaler(feature_range=(0, 1))
df_1 = scaler.fit_transform(a2)


C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
df_1

array([[0.04672897],
       [0.04672897],
       [0.03738318],
       ...,
       [0.24299065],
       [0.38317757],
       [0.39252336]])

In [10]:
data1 = df.drop(['Appliances'],axis=1)
df_2 = scaler.fit_transform(data1) # features 


In [11]:
df_2

array([[0.42857143, 0.32734952, 0.56618659, ..., 0.        , 0.        ,
        0.        ],
       [0.42857143, 0.32734952, 0.54132648, ..., 0.        , 0.        ,
        0.        ],
       [0.42857143, 0.32734952, 0.53050179, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.14285714, 0.91974657, 0.53866618, ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.91974657, 0.54949087, ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.91974657, 0.53875791, ..., 0.        , 0.        ,
        0.        ]])

In [12]:
x = df_2
y = df_1

In [13]:
Ytrainsize = int(len(y) * 0.7)
Ytestsize = (len(y) - Ytrainsize)
train_Y, test_Y = y[0:Ytrainsize,:], y[Ytrainsize:len(df_1),:]
print(len(train_Y) , len(test_Y))

Xtrainsize = int(len(x) * 0.7)


Xtestsize = (len(x) - Xtrainsize)
train_X, test_X = x[0:Xtrainsize,:], x[Xtrainsize:len(df_2),:]
print(len(train_X) , len(test_X))

13814 5921
13814 5921


In [14]:
train_Y=train_Y.ravel()


In [14]:
mlp = MLPRegressor(hidden_layer_sizes=(38,38,10,), activation='tanh', solver='sgd', alpha=0.15, batch_size=180,  max_iter=150,  random_state=None,  verbose=False, warm_start=False, momentum=0.9,learning_rate_init=0.01, validation_fraction=0.1,epsilon=1e-08)
mlp.fit(train_X, train_Y)

MLPRegressor(activation='tanh', alpha=0.15, batch_size=180, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(38, 38, 10), learning_rate='constant',
       learning_rate_init=0.01, max_iter=150, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [20]:
ypredtest = mlp.predict(test_X)

In [21]:
print("Scores for test")
scores("MLP - Scaled",test_Y,ypredtest)

Scores for test
The scores for the model :  MLP - Scaled
MAE    :     0.046838494426055415
RMSE   :     0.0816844647705519
R2     :     0.08906230082594346
MAPE   :     89.05102667269009


In [22]:
ypredtrain = mlp.predict(train_X)

In [23]:
print("Scores for train")
scores("MLP - Scaled",train_Y,ypredtrain)

Scores for train
The scores for the model :  MLP - Scaled
MAE    :     0.053188756266624906
RMSE   :     0.09313050036384826
R2     :     0.13028984527973553
MAPE   :     inf


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


MLP on Unscaled Data

In [24]:
y=df.Appliances.values
y = y.reshape(len(y),1)
y

array([[ 60],
       [ 60],
       [ 50],
       ...,
       [270],
       [420],
       [430]], dtype=int64)

In [25]:
x=data1.values

In [26]:
Ytrainsize = int(len(y) * 0.7)
Ytestsize = (len(y) - Ytrainsize)
train_Y, test_Y = y[0:Ytrainsize,:], y[Ytrainsize:len(df_1),:]
print(len(train_Y) , len(test_Y))

Xtrainsize = int(len(x) * 0.7)


Xtestsize = (len(x) - Xtrainsize)
train_X, test_X = x[0:Xtrainsize,:], x[Xtrainsize:len(df_2),:]
print(len(train_X) , len(test_X))

13814 5921
13814 5921


In [27]:
train_Y=train_Y.ravel()

In [28]:
mlp = MLPRegressor(hidden_layer_sizes=(38,38,10,), activation='tanh', solver='sgd', alpha=0.15, batch_size=180,  max_iter=150,  random_state=None,  verbose=False, warm_start=False, momentum=0.9,learning_rate_init=0.01, validation_fraction=0.1,epsilon=1e-08)
mlp.fit(train_X, train_Y)

MLPRegressor(activation='tanh', alpha=0.15, batch_size=180, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(38, 38, 10), learning_rate='constant',
       learning_rate_init=0.01, max_iter=150, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [32]:
ypredtest1 = mlp.predict(test_X)

In [30]:
test_Y

array([[120],
       [150],
       [100],
       ...,
       [270],
       [420],
       [430]], dtype=int64)

In [33]:
ypredtest1


array([100.85483547, 100.85483547, 100.85483547, ..., 100.85483547,
       100.85483547, 100.85483547])

In [34]:
rmse_test = sqrt(mean_squared_error(test_Y,ypredtest1))
print('Test RMSE: %.3f' % rmse_test)

Test RMSE: 91.751


Running MLP on Features with Rank 1 --- Boruta

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,40,19.890000,46.026667,19.200000,44.500000,19.790000,44.933333,18.890000,45.730000,17.133333,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,50,19.890000,45.766667,19.200000,44.500000,19.790000,44.900000,18.890000,45.790000,17.100000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,50,19.856667,45.560000,19.200000,44.500000,19.730000,44.900000,18.890000,45.863333,17.100000,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,40,19.790000,45.597500,19.200000,44.433333,19.730000,44.790000,18.890000,45.790000,17.166667,...,18,11,0,66000,0,0,0,0,0,0


In [36]:
data1.head()

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,17,11,0,63600,0,0,0,0,0,0


Columns with Rank 1 by Boruta

 Top 17 features:
       features  rank
0        lights     1
1   Press_mm_hg     1
2         month     1
3            T9     1
4            T8     1
5          RH_7     1
6          RH_6     1
7     Windspeed     1
8          RH_5     1
9     Tdewpoint     1
10         RH_4     1
11         RH_3     1
12           T3     1
13         RH_2     1
14         hour     1
15          NSM     1
16           T5     1


In [15]:
data1.columns.values

array(['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
       'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
       'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'month', 'hour', 'day',
       'weekendstatus', 'NSM', 'Friday', 'Saturday', 'Sunday', 'Thursday',
       'Tuesday', 'Wednesday'], dtype=object)

In [16]:
data2 = data1

In [17]:
data2 = data2.drop(['T1'],axis=1)

In [18]:
data2 = data2.drop(['RH_1'],axis=1)

In [19]:
data2 = data2.drop(['T2'],axis=1)

In [20]:
data2 = data2.drop(['T4'],axis=1)

In [21]:
data2 = data2.drop(['T6'],axis=1)
data2 = data2.drop(['T7'],axis=1)

In [22]:
data2 = data2.drop(['RH_9'],axis=1)

In [23]:
data2 = data2.drop(['T_out'],axis=1)

In [24]:
data2 = data2.drop(['RH_out'],axis=1)
data2 = data2.drop(['Visibility'],axis=1)
data2 = data2.drop(['rv1'],axis=1)
data2 = data2.drop(['rv2'],axis=1)
data2 = data2.drop(['day'],axis=1)
data2 = data2.drop(['weekendstatus'],axis=1)
data2 = data2.drop(['Friday'],axis=1)
data2 = data2.drop(['Saturday'],axis=1)
data2 = data2.drop(['Sunday'],axis=1)
data2 = data2.drop(['Thursday'],axis=1)
data2 = data2.drop(['Tuesday'],axis=1)
data2 = data2.drop(['Wednesday'],axis=1)
data2 = data2.drop(['RH_8'],axis=1)

In [25]:
data2.columns.values

array(['lights', 'RH_2', 'T3', 'RH_3', 'RH_4', 'T5', 'RH_5', 'RH_6',
       'RH_7', 'T8', 'T9', 'Press_mm_hg', 'Windspeed', 'Tdewpoint',
       'month', 'hour', 'NSM'], dtype=object)

In [26]:
df_3 = scaler.fit_transform(data2) # borutafeatures 



In [27]:
df_3

array([[0.42857143, 0.68403787, 0.21518777, ..., 0.        , 0.73913043,
        0.71328671],
       [0.42857143, 0.68213984, 0.21518777, ..., 0.        , 0.73913043,
        0.72027972],
       [0.42857143, 0.67944512, 0.21518777, ..., 0.        , 0.73913043,
        0.72727273],
       ...,
       [0.14285714, 0.62719762, 0.8183782 , ..., 1.        , 0.73913043,
        0.74125874],
       [0.14285714, 0.63471741, 0.80508475, ..., 1.        , 0.73913043,
        0.74825175],
       [0.14285714, 0.63290173, 0.79954581, ..., 1.        , 0.7826087 ,
        0.75524476]])

In [28]:
x = df_3
y = df_1

In [29]:
Ytrainsize = int(len(y) * 0.7)
Ytestsize = (len(y) - Ytrainsize)
train_Y, test_Y = y[0:Ytrainsize,:], y[Ytrainsize:len(df_1),:]
print(len(train_Y) , len(test_Y))

Xtrainsize = int(len(x) * 0.7)


Xtestsize = (len(x) - Xtrainsize)
train_X, test_X = x[0:Xtrainsize,:], x[Xtrainsize:len(df_2),:]
print(len(train_X) , len(test_X))

13814 5921
13814 5921


In [30]:
train_Y=train_Y.ravel()

In [79]:
mlp.fit(train_X, train_Y)

MLPRegressor(activation='tanh', alpha=0.15, batch_size=180, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(38, 38, 10), learning_rate='constant',
       learning_rate_init=0.01, max_iter=150, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [80]:
ypredtrainb = mlp.predict(train_X)

In [81]:
ypredtestb = mlp.predict(test_X)

In [84]:
print("Scores for test with Boruta Features")
scores("MLP - Scaled",test_Y,ypredtestb)

Scores for test with Boruta Features
The scores for the model :  MLP - Scaled
MAE    :     0.04505609007153024
RMSE   :     0.08184386644342663
R2     :     0.08550356606844967
MAPE   :     82.67522329983609


In [83]:
print("Scores for test")
scores("MLP - Scaled",test_Y,ypredtest)

Scores for test
The scores for the model :  MLP - Scaled
MAE    :     0.046838494426055415
RMSE   :     0.0816844647705519
R2     :     0.08906230082594346
MAPE   :     89.05102667269009


In [85]:
print("Scores for train with Boruta Features")
scores("MLP - Scaled",train_Y,ypredtrainb)

Scores for train with Boruta Features
The scores for the model :  MLP - Scaled
MAE    :     0.05275645221641778
RMSE   :     0.09401173682997147
R2     :     0.11375291309266078
MAPE   :     inf


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


In [86]:
print("Scores for train")
scores("MLP - Scaled",train_Y,ypredtrain)

Scores for train
The scores for the model :  MLP - Scaled
MAE    :     0.053188756266624906
RMSE   :     0.09313050036384826
R2     :     0.13028984527973553
MAPE   :     inf


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


In [31]:
 mlpfinal = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.15, batch_size='auto',  max_iter=200,  random_state=None,  verbose=False, warm_start=False, momentum=0.9,learning_rate_init=0.01, validation_fraction=0.1,epsilon=1e-08)


In [32]:
mlpfinal.fit(train_X, train_Y)

MLPRegressor(activation='relu', alpha=0.15, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.01, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [34]:
ypredtrainb = mlpfinal.predict(train_X)
ypredtestb = mlpfinal.predict(test_X)

In [35]:
print("Scores for test with Boruta Features")
scores("MLP - Scaled",test_Y,ypredtestb)

Scores for test with Boruta Features
The scores for the model :  MLP - Scaled
MAE    :     0.04002498446365815
RMSE   :     0.08168043472311416
R2     :     0.08915218405135616
MAPE   :     69.17064426267237


In [36]:
print("Scores for train with Boruta Features")
scores("MLP - Scaled",train_Y,ypredtrainb)

Scores for train with Boruta Features
The scores for the model :  MLP - Scaled
MAE    :     0.04628927278846125
RMSE   :     0.09402470146180283
R2     :     0.11350846149901017
MAPE   :     inf


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
